In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from scipy.optimize import minimize
from scipy import stats
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from colorama import Fore, Style
from IPython.display import clear_output
from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
# Create output folders
output_folder = 'output'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Create separate analysis output folders
analysis_output_folder = 'analysis_output'
os.makedirs(analysis_output_folder, exist_ok=True)

physical_analysis_output_folder = 'analysis_output/physical'
os.makedirs(physical_analysis_output_folder, exist_ok=True)

fitness_analysis_output_folder = 'analysis_output/fitness'
os.makedirs(fitness_analysis_output_folder, exist_ok=True)

bia_analysis_output_folder = 'analysis_output/bia'
os.makedirs(bia_analysis_output_folder, exist_ok=True)

child_info_analysis_output_folder = 'analysis_output/child_info'
os.makedirs(child_info_analysis_output_folder, exist_ok=True)

actigraphy_analysis_output_folder = 'analysis_output/actigraphy'
os.makedirs(actigraphy_analysis_output_folder, exist_ok=True)

# Load data functions
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    stats, indexes = zip(*results)
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df

# Set display all columns in dataframes property
pd.options.display.max_columns = None

In [ ]:
# Load data
train_data = pd.read_csv('input/train.csv')
test_data = pd.read_csv('input/test.csv')
sample_data = pd.read_csv('input/sample_submission.csv')

train_ts_data = load_time_series("input/series_train.parquet")
test_ts_data = load_time_series("input/series_test.parquet")

In [ ]:
# Remove id column from time series data
time_series_columns = train_ts_data.columns.tolist()
time_series_columns.remove("id")

# Merge data
train_data = pd.merge(train_data, train_ts_data, how="left", on='id')
test_data = pd.merge(test_data, test_ts_data, how="left", on='id')
train_data = train_data.drop('id', axis=1)
test_data = test_data.drop('id', axis=1)

train_data.head()

In [ ]:
# Feature engineering
train_data = train_data.copy()

# Combine all grip strength
train_data['FGC-FGC_GS'] = train_data['FGC-FGC_GSD_Zone'] + train_data['FGC-FGC_GSND_Zone']

# Combine all sit and reach
train_data['FGC-FGC_SR'] = train_data['FGC-FGC_SRL_Zone'] + train_data['FGC-FGC_SRR_Zone']

# Create a fitness score by adding the zone fitness data
train_data['fitness_score'] = train_data['FGC-FGC_GS'] + train_data['FGC-FGC_SR'] + train_data['FGC-FGC_CU_Zone'] + train_data['FGC-FGC_PU_Zone'] + train_data['FGC-FGC_TL_Zone']

# Combine PAQ_A-PAQ_A_Total and PAQ_C-PAQ_C_Total into one column
train_data['PAQ_Total'] = train_data['PAQ_A-PAQ_A_Total'].combine_first(train_data['PAQ_C-PAQ_C_Total'])

# Create interaction features
train_data['BIA_Fat_X_PCIAT_Total'] = train_data['BIA-BIA_Fat'] * train_data['PCIAT-PCIAT_Total']
train_data['Physical_Weight_X_BIA_BMI'] = train_data['Physical-Weight'] * train_data['BIA-BIA_BMI']
train_data['CGAS_Score_X_PCIAT_Total'] = train_data['CGAS-CGAS_Score'] * train_data['PCIAT-PCIAT_Total']
train_data['BIA_Fat_X_Physical_Weight'] = train_data['BIA-BIA_Fat'] * train_data['Physical-Weight']
train_data['BIA_BMI_X_PCIAT_Total'] = train_data['BIA-BIA_BMI'] * train_data['PCIAT-PCIAT_Total']
train_data['Physical_Weight_X_PCIAT_Total'] = train_data['Physical-Weight'] * train_data['PCIAT-PCIAT_Total']

# Combine identical actigraphy stats based on correlation analysis
def combine_identical_features(df, columns_to_combine, new_column_name):
    # Verify if identical
    is_identical = df[columns_to_combine].nunique().eq(1).all()
    
    if is_identical:
        # If they are identical, we can just take the first column and rename it
        df[new_column_name] = df[columns_to_combine[0]]
        
        # Drop the original columns
        df = df.drop(columns=columns_to_combine)
        
        print(f"Columns {columns_to_combine} have been combined into {new_column_name}")
    else:
        # If not exactly identical, take the mean
        print("Warning: The columns are not identical. Taking the average instead.")
        df[new_column_name] = df[columns_to_combine].mean(axis=1)
        
        # Drop the original columns
        df = df.drop(columns=columns_to_combine)
    
    return df

# List of columns to combine
columns_to_combine = ['stat_0', 'stat_1', 'stat_2', 'stat_3', 'stat_4', 'stat_5', 
                      'stat_6', 'stat_7', 'stat_8', 'stat_9', 'stat_10', 'stat_11']

# Apply to both train and test datasets
train_data = combine_identical_features(train_data, columns_to_combine, 'combined_actigraphy_stat')

# Polynomial function
def create_polynomial_features(df, features, degrees):
    """
    Create polynomial features for specified columns, handling NaN values.
    
    :param df: DataFrame
    :param features: List of feature names to create polynomials for
    :param degrees: Dictionary of feature names and their corresponding degrees
    :return: DataFrame with added polynomial features
    """
    # Create a copy of the dataframe to avoid modifying the original
    df_poly = df.copy()
    
    for feature in features:
        max_degree = degrees.get(feature, 2)  # Default to degree 2 if not specified
        
        # Check for NaN values
        nan_count = df_poly[feature].isna().sum()
        if nan_count > 0:
            print(f"Warning: {nan_count} NaN values found in {feature}. These will be imputed.")
        
        # Impute NaN values with median
        imputer = SimpleImputer(strategy='median')
        df_poly[feature] = imputer.fit_transform(df_poly[[feature]])
        
        # Create polynomial features
        poly = PolynomialFeatures(degree=max_degree, include_bias=False)
        feature_poly = poly.fit_transform(df_poly[[feature]])
        
        # Create new column names
        feature_names = [f"{feature}^{i}" for i in range(1, max_degree + 1)]
        
        # Add new columns to the dataframe
        for i, name in enumerate(feature_names):
            df_poly[name] = feature_poly[:, i]
    
    return df_poly

# Specify features and their degrees
features_to_transform = ['BIA-BIA_Fat', 'Physical-Weight']
degrees = {'BIA-BIA_Fat': 3, 'Physical-Weight': 2}

# Apply to both train and test datasets
train_data = create_polynomial_features(train_data, features_to_transform, degrees)

In [ ]:
train_data.head()

In [ ]:
# Skew removal for some BIA columns
skewed_columns = [
    'BIA-BIA_BMC', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_Fat',
    'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 
    'BIA-BIA_TBW', 'CGAS-CGAS_Score', 'stat_23', 'stat_35', 'stat_38', 'stat_40', 'stat_47',
    'stat_54', 'stat_66', 'stat_78', 'stat_80', 'stat_88', 'stat_90'
]
lambda_params = {}

# Define the box-cox function to remove skew
def box_cox_transform(df, column):
    # Create a copy of the dataframe
    df_copy = df.copy()
    
    # Drop NaN values for the specific column
    df_copy = df_copy.dropna(subset=[column])
    
    # Ensure all values are positive
    min_value = df_copy[column].min()
    if min_value <= 0:
        df_copy[column] = df_copy[column] - min_value + 1  # Add 1 to ensure all values are positive
    
    # Perform Box-Cox transformation
    df_copy[f'{column}_boxcox'], lambda_param = stats.boxcox(df_copy[column])
    
    print(f"Transforming column: {column}")
    print(f"Optimal lambda for Box-Cox transformation: {lambda_param}")
    print(f"Number of rows before transformation: {len(df)}")
    print(f"Number of rows after removing NaN values: {len(df_copy)}")
    
    return df_copy, lambda_param

for column in skewed_columns:
    transformed_train_data, lambda_params[column] = box_cox_transform(train_data, column)
    # Update only the new transformed column in the original dataframe
    train_data[f'{column}_boxcox'] = transformed_train_data[f'{column}_boxcox']

In [8]:
# Isolate the physical attribute columns and some contextual columns for analysis
physical_columns = [
    'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex', 'Physical_Weight_X_PCIAT_Total',
    'CGAS-Season', 'CGAS-CGAS_Score_boxcox', 'Physical-Season', 'Physical-BMI',
    'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference', 'BIA_Fat_X_Physical_Weight',
    'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP', 'BIA_BMI_X_PCIAT_Total',
    'fitness_score', 'BIA-BIA_Frame_num', 'BIA-BIA_BMI', 'PreInt_EduHx-computerinternet_hoursday',
    'PCIAT-PCIAT_Total', 'BIA_Fat_X_PCIAT_Total', 'Physical_Weight_X_BIA_BMI', 'CGAS_Score_X_PCIAT_Total',
    'BIA-BIA_Fat^1', 'BIA-BIA_Fat^2', 'BIA-BIA_Fat^3', 'Physical-Weight^1',
    'Physical-Weight^2', 'sii'
]

# Isolate the fitness attributes
# Removed columns: 'FGC-FGC_CU' 'FGC-FGC_PU', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_SRL' 
# 'FGC-FGC_GSND_Zone' 'FGC-FGC_GSND' 'FGC-FGC_GSD' 'FGC-FGC_GSD_Zone' 'Fitness_Endurance-Time_Sec'
fitness_columns = [
    'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins', 'Physical_Weight_X_PCIAT_Total',
    'FGC-Season', 'FGC-FGC_CU_Zone', 'FGC-FGC_SR', 'FGC-FGC_PU_Zone', 'FGC-FGC_TL', 'BIA_Fat_X_Physical_Weight',
    'FGC-FGC_TL_Zone', 'FGC-FGC_GS', 'fitness_score', 'BIA-BIA_BMI', 'Physical-BMI', 'BIA_BMI_X_PCIAT_Total',
    'BIA-BIA_Frame_num', 'PreInt_EduHx-computerinternet_hoursday', 'PCIAT-PCIAT_Total', 'BIA_Fat_X_PCIAT_Total',
    'Physical_Weight_X_BIA_BMI', 'CGAS_Score_X_PCIAT_Total', 'BIA-BIA_Fat^1', 'BIA-BIA_Fat^2', 'BIA-BIA_Fat^3',
    'Physical-Weight^1', 'Physical-Weight^2', 'sii'
]

# Isolate the BIA attributes
bia_columns = [
    'BIA-Season', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_Frame_num', 'BIA-BIA_BMC_boxcox', 'BIA_Fat_X_Physical_Weight',
    'BIA-BIA_BMI', 'BIA-BIA_BMR_boxcox', 'BIA-BIA_DEE_boxcox', 'BIA-BIA_ECW_boxcox', 'BIA-BIA_FFM_boxcox', 'BIA-BIA_FFMI_boxcox',
    'BIA-BIA_FMI_boxcox', 'BIA-BIA_Fat_boxcox', 'BIA-BIA_ICW_boxcox', 'BIA-BIA_LDM_boxcox', 'BIA-BIA_LST_boxcox', 'BIA-BIA_TBW_boxcox',
    'fitness_score', 'Physical-BMI', 'PreInt_EduHx-computerinternet_hoursday', 'PCIAT-PCIAT_Total', 'BIA_Fat_X_PCIAT_Total',
    'Physical_Weight_X_BIA_BMI', 'CGAS_Score_X_PCIAT_Total', 'BIA_BMI_X_PCIAT_Total', 'Physical_Weight_X_PCIAT_Total',
    'BIA-BIA_Fat^1', 'BIA-BIA_Fat^2', 'BIA-BIA_Fat^3', 'Physical-Weight^1', 'Physical-Weight^2', 'sii'
]

# Isolate the PAQ, PCIAT, and SDS
child_info_columns = [
    'PreInt_EduHx-computerinternet_hoursday', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-PAQ_C_Total',
    'PAQ_Total', 'PCIAT-Season', 'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02', 'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04',
    'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'BIA_Fat_X_Physical_Weight',
    'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'BIA_BMI_X_PCIAT_Total',
    'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19', 'Physical_Weight_X_PCIAT_Total',
    'PCIAT-PCIAT_20', 'PCIAT-PCIAT_Total', 'SDS-Season', 'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T',
    'PreInt_EduHx-Season', 'PreInt_EduHx-computerinternet_hoursday', 'BIA-BIA_BMI', 'fitness_score', 'BIA_Fat_X_PCIAT_Total',
    'Physical-BMI', 'Physical_Weight_X_BIA_BMI', 'CGAS_Score_X_PCIAT_Total', 'BIA-BIA_Fat^1',
    'BIA-BIA_Fat^2', 'BIA-BIA_Fat^3', 'Physical-Weight^1', 'Physical-Weight^2', 'sii'
]

# Isolate the Actigraphy data
# removed columns: 'stat_41', 'stat_42', 'stat_39','stat_92_boxcox' 'stat_0', 'stat_1', 'stat_2', 'stat_3', 'stat_4', 'stat_5', 'stat_6', 'stat_7', 
# 'stat_8', 'stat_9', 'stat_10','stat_11'
actigraphy_columns = [
    'combined_actigraphy_stat', 'stat_12', 'stat_13', 'stat_14', 'stat_15', 'stat_16', 'stat_17', 'stat_18', 'stat_19', 'stat_20',
    'stat_21', 'stat_22', 'stat_23_boxcox', 'stat_24', 'stat_25', 'stat_26', 'stat_27', 'stat_28', 'stat_29', 'stat_30',
    'stat_31', 'stat_32', 'stat_33', 'stat_34', 'stat_35_boxcox', 'stat_36', 'stat_37', 'stat_38_boxcox', 'stat_40_boxcox',
    'stat_43', 'stat_44', 'stat_45', 'stat_46', 'stat_47_boxcox', 'stat_48', 'stat_49', 'stat_50', 'BIA_Fat_X_Physical_Weight',
    'stat_51', 'stat_52', 'stat_53', 'stat_54_boxcox', 'stat_55', 'stat_56', 'stat_57', 'stat_58', 'stat_59', 'stat_60',
    'stat_61', 'stat_62', 'stat_63', 'stat_64', 'stat_65', 'stat_66_boxcox', 'stat_67', 'stat_68', 'stat_69', 'stat_70',
    'stat_71', 'stat_72', 'stat_73', 'stat_74', 'stat_75', 'stat_76', 'stat_77', 'stat_78_boxcox', 'stat_79', 'stat_80_boxcox',
    'stat_81', 'stat_82', 'stat_83', 'stat_84', 'stat_85', 'stat_86', 'stat_87', 'stat_88_boxcox', 'stat_89', 'stat_90_boxcox',
    'stat_91', 'stat_93', 'stat_94', 'stat_95', 'PreInt_EduHx-computerinternet_hoursday', 'CGAS_Score_X_PCIAT_Total', 'BIA_BMI_X_PCIAT_Total',
    'BIA-BIA_Frame_num', 'SDS-SDS_Total_T', 'BIA-BIA_BMI', 'Physical-BMI', 'BIA_Fat_X_PCIAT_Total', 'Physical_Weight_X_PCIAT_Total', 'sii'
]

In [ ]:
# Function to analyze columns
def analyze_column(column):
    total_count = len(train_data)
    missing_count = train_data[column].isnull().sum()
    missing_percentage = (missing_count / total_count) * 100
    unique_values = train_data[column].nunique()
    
    if pd.api.types.is_numeric_dtype(train_data[column]):
        mean_value = train_data[column].mean()
        median_value = train_data[column].median()
        std_dev = train_data[column].std()
        min_value = train_data[column].min()
        max_value = train_data[column].max()
        return {
            "Column": column,
            "Total Count": total_count,
            "Missing Count": missing_count,
            "Missing Percentage": f"{missing_percentage:.2f}%",
            "Unique Values": unique_values,
            "Data Type": train_data[column].dtype,
            "Mean": mean_value,
            "Median": median_value,
            "Standard Deviation": std_dev,
            "Minimum": min_value,
            "Maximum": max_value
        }
    else:
        top_values = train_data[column].value_counts().head(3).to_dict()
        return {
            "Column": column,
            "Total Count": total_count,
            "Missing Count": missing_count,
            "Missing Percentage": f"{missing_percentage:.2f}%",
            "Unique Values": unique_values,
            "Data Type": train_data[column].dtype,
            "Top 3 Values": top_values
        }

# Physical column profiles        
physical_column_profiles = [analyze_column(col) for col in physical_columns]
physical_column_profiles_df = pd.DataFrame(physical_column_profiles)

# Save column profiles to CSV
physical_column_profiles_df.to_csv(os.path.join(physical_analysis_output_folder, 'physical_column_profiles.csv'), index=False)
print(f"Column profiles saved to {os.path.join(physical_analysis_output_folder, 'physical_column_profiles.csv')}")

# Fitness column profiles
fitness_column_profiles = [analyze_column(col) for col in fitness_columns]
fitness_column_profiles_df = pd.DataFrame(fitness_column_profiles)

# Save column profiles to CSV
fitness_column_profiles_df.to_csv(os.path.join(analysis_output_folder, 'fitness_column_profiles.csv'), index=False)
print(f"Column profiles saved to {os.path.join(analysis_output_folder, 'fitness_column_profiles.csv')}")

# BIA column profiles
bia_column_profiles = [analyze_column(col) for col in bia_columns]
bia_column_profiles_df = pd.DataFrame(bia_column_profiles)

# Save column profiles to CSV
bia_column_profiles_df.to_csv(os.path.join(analysis_output_folder, 'bia_column_profiles.csv'), index=False)
print(f"Column profiles saved to {os.path.join(analysis_output_folder, 'bia_column_profiles.csv')}")

# Child info column profiles
child_info_column_profiles = [analyze_column(col) for col in child_info_columns]
child_info_column_profiles_df = pd.DataFrame(child_info_column_profiles)

# Save column profiles to CSV
child_info_column_profiles_df.to_csv(os.path.join(analysis_output_folder, 'child_info_column_profiles.csv'), index=False)
print(f"Column profiles saved to {os.path.join(analysis_output_folder, 'child_info_column_profiles.csv')}")

# Actigraphy info column profiles
actigraphy_column_profiles = [analyze_column(col) for col in actigraphy_columns]
actigraphy_column_profiles_df = pd.DataFrame(actigraphy_column_profiles)

# Save column profiles to CSV
actigraphy_column_profiles_df.to_csv(os.path.join(analysis_output_folder, 'actigraphy_column_profiles.csv'), index=False)
print(f"Column profiles saved to {os.path.join(analysis_output_folder, 'actigraphy_column_profiles.csv')}")

In [ ]:
# Visualize missing data
# Physical columns
plt.figure(figsize=(12, 6))
sns.heatmap(train_data[physical_columns].isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title('Missing Data in Physical Attribute Columns')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.tight_layout()
plt.savefig(os.path.join(physical_analysis_output_folder, 'physical_missing_data_heatmap.png'))
plt.close()
print(f"Missing data heatmap saved to {os.path.join(physical_analysis_output_folder, 'physical_missing_data_heatmap.png')}")

# Fitness columns
plt.figure(figsize=(12, 6))
sns.heatmap(train_data[fitness_columns].isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title('Missing Data in Fitness Attribute Columns')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.tight_layout()
plt.savefig(os.path.join(fitness_analysis_output_folder, 'fitness_missing_data_heatmap.png'))
plt.close()
print(f"Missing data heatmap saved to {os.path.join(fitness_analysis_output_folder, 'fitness_missing_data_heatmap.png')}")

# BIA columns
plt.figure(figsize=(12, 6))
sns.heatmap(train_data[bia_columns].isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title('Missing Data in BIA Attribute Columns')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.tight_layout()
plt.savefig(os.path.join(bia_analysis_output_folder, 'bia_missing_data_heatmap.png'))
plt.close()
print(f"Missing data heatmap saved to {os.path.join(bia_analysis_output_folder, 'bia_missing_data_heatmap.png')}")

# Child info columns
plt.figure(figsize=(12, 6))
sns.heatmap(train_data[child_info_columns].isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title('Missing Data in Child info Attribute Columns')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.tight_layout()
plt.savefig(os.path.join(child_info_analysis_output_folder, 'child_info_missing_data_heatmap.png'))
plt.close()
print(f"Missing data heatmap saved to {os.path.join(child_info_analysis_output_folder, 'child_info_missing_data_heatmap.png')}")

# Actigraphy info columns
plt.figure(figsize=(40, 6))
sns.heatmap(train_data[actigraphy_columns].isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title('Missing Data in Actigraphy info Attribute Columns')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.tight_layout()
plt.savefig(os.path.join(actigraphy_analysis_output_folder, 'actigraphy_missing_data_heatmap.png'))
plt.close()
print(f"Missing data heatmap saved to {os.path.join(actigraphy_analysis_output_folder, 'actigraphy_missing_data_heatmap.png')}")

In [ ]:
# Correlation matrix for physical numeric columns
physical_numeric_columns = train_data[physical_columns].select_dtypes(include=[np.number]).columns
physical_correlation_matrix = train_data[physical_numeric_columns].corr()

plt.figure(figsize=(20, 18))
sns.heatmap(physical_correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Numeric Physical Attributes')
plt.tight_layout()
plt.savefig(os.path.join(physical_analysis_output_folder, 'physical_correlation_matrix.png'))
plt.close()
print(f"Physical correlation matrix saved to {os.path.join(physical_analysis_output_folder, 'physical_correlation_matrix.png')}")

# Correlation matrix for fitness numeric columns
fitness_numeric_columns = train_data[fitness_columns].select_dtypes(include=[np.number]).columns
fitness_correlation_matrix = train_data[fitness_numeric_columns].corr()

plt.figure(figsize=(20, 18))
sns.heatmap(fitness_correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Numeric Fitness Attributes')
plt.tight_layout()
plt.savefig(os.path.join(fitness_analysis_output_folder, 'fitness_correlation_matrix.png'))
plt.close()
print(f"Fitness correlation matrix saved to {os.path.join(fitness_analysis_output_folder, 'fitness_correlation_matrix.png')}")

# Correlation matrix for bia numeric columns
bia_numeric_columns = train_data[bia_columns].select_dtypes(include=[np.number]).columns
bia_correlation_matrix = train_data[bia_numeric_columns].corr()

plt.figure(figsize=(20, 18))
sns.heatmap(bia_correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Numeric BIA Attributes')
plt.tight_layout()
plt.savefig(os.path.join(bia_analysis_output_folder, 'bia_correlation_matrix.png'))
plt.close()
print(f"BIA correlation matrix saved to {os.path.join(bia_analysis_output_folder, 'BIA_correlation_matrix.png')}")

# Correlation matrix for child info numeric columns
child_info_numeric_columns = train_data[child_info_columns].select_dtypes(include=[np.number]).columns
child_info_correlation_matrix = train_data[child_info_numeric_columns].corr()

plt.figure(figsize=(24, 22))
sns.heatmap(child_info_correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Numeric Child info Attributes')
plt.tight_layout()
plt.savefig(os.path.join(child_info_analysis_output_folder, 'child_info_correlation_matrix.png'))
plt.close()
print(f"BIA correlation matrix saved to {os.path.join(child_info_analysis_output_folder, 'child_info_correlation_matrix.png')}")

# Correlation matrix for actigraphy numeric columns
actigraphy_numeric_columns = train_data[actigraphy_columns].select_dtypes(include=[np.number]).columns
actigraphy_correlation_matrix = train_data[actigraphy_numeric_columns].corr()

plt.figure(figsize=(80, 78))
sns.heatmap(actigraphy_correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Numeric Actigraphy Attributes')
plt.tight_layout()
plt.savefig(os.path.join(actigraphy_analysis_output_folder, 'actigraphy_correlation_matrix.png'))
plt.close()
print(f"BIA correlation matrix saved to {os.path.join(actigraphy_analysis_output_folder, 'actigraphy_correlation_matrix.png')}")

In [ ]:
# Combine all columns into a single list
all_columns = []
all_columns.extend(physical_numeric_columns)
all_columns.extend(fitness_numeric_columns)
all_columns.extend(bia_numeric_columns)
all_columns.extend(child_info_numeric_columns)
all_columns.extend(actigraphy_numeric_columns)

# Create a tqdm progress bar
with tqdm(total=len(all_columns), desc="Creating distribution plots") as pbar:
    # Distribution plots for physical numeric columns
    for column in physical_numeric_columns:
        plt.figure(figsize=(10, 6))
        sns.histplot(train_data[column].dropna(), kde=True)
        plt.title(f'Distribution of physical {column}')
        plt.xlabel(column)
        plt.ylabel('Count')
        plt.tight_layout()
        plt.savefig(os.path.join(physical_analysis_output_folder, f'physical_{column}_distribution.png'))
        plt.close()
        pbar.update(1)

    # Distribution plots for fitness numeric columns
    for column in fitness_numeric_columns:
        plt.figure(figsize=(10, 6))
        sns.histplot(train_data[column].dropna(), kde=True)
        plt.title(f'Distribution of fitness {column}')
        plt.xlabel(column)
        plt.ylabel('Count')
        plt.tight_layout()
        plt.savefig(os.path.join(fitness_analysis_output_folder, f'fitness_{column}_distribution.png'))
        plt.close()
        pbar.update(1)

    # Distribution plots for BIA numeric columns
    for column in bia_numeric_columns:
        plt.figure(figsize=(10, 6))
        sns.histplot(train_data[column].dropna(), kde=True)
        plt.title(f'Distribution of bia {column}')
        plt.xlabel(column)
        plt.ylabel('Count')
        plt.tight_layout()
        plt.savefig(os.path.join(bia_analysis_output_folder, f'bia_{column}_distribution.png'))
        plt.close()
        pbar.update(1)
        
    # Distribution plots for Child info numeric columns
    for column in child_info_numeric_columns:
        plt.figure(figsize=(10, 6))
        sns.histplot(train_data[column].dropna(), kde=True)
        plt.title(f'Distribution of child info {column}')
        plt.xlabel(column)
        plt.ylabel('Count')
        plt.tight_layout()
        plt.savefig(os.path.join(child_info_analysis_output_folder, f'child_info_{column}_distribution.png'))
        plt.close()
        pbar.update(1)
        
    # Distribution plots for actigraphy numeric columns
    for column in actigraphy_numeric_columns:
        plt.figure(figsize=(10, 6))
        sns.histplot(train_data[column].dropna(), kde=True)
        plt.title(f'Distribution of actigraphy {column}')
        plt.xlabel(column)
        plt.ylabel('Count')
        plt.tight_layout()
        plt.savefig(os.path.join(actigraphy_analysis_output_folder, f'actigraphy_{column}_distribution.png'))
        plt.close()
        pbar.update(1)

print("All analyses completed and saved to the 'analysis_output' folder.")

In [12]:
# Supplement missing data with data from WHO
# Load who data and group
def load_who_bmi_data(file_path):
    who_data = pd.read_csv(file_path)
    who_data = who_data.groupby(['age', 'sex']).agg({
        'L': 'mean', 'mean_bmi': 'mean', 'S': 'mean'
    }).reset_index()
    who_data = who_data.set_index(['sex', 'age'])
    return who_data

def load_who_height_data(file_path):
    who_data = pd.read_csv(file_path)
    who_data = who_data.groupby(['age', 'sex']).agg({
        'mean_height': 'mean'
    }).reset_index()
    who_data = who_data.set_index(['sex', 'age'])
    return who_data

who_bmi_data = load_who_bmi_data('supplemental_data/bmi_for_age_5_to_19.csv')
who_height_data = load_who_height_data('supplemental_data/height_for_age_5_to_19.csv')

In [13]:
# Defining functions to Impute with data from WHO
def get_who_stats(age, sex, data_type='bmi'):
    try:
        if data_type == 'bmi':
            stats = who_bmi_data.loc[(sex, age), ['mean_bmi', 'S']]
            return stats['mean_bmi'], stats['S']
        elif data_type == 'height':
            stats = who_height_data.loc[(sex, age), 'mean_height']
            return stats
    except KeyError:
        return None, None if data_type == 'bmi' else None
    
def impute_bmi(age, sex):
    mean_bmi, sd = get_who_stats(age, sex, 'bmi')
    if mean_bmi is not None and sd is not None:
        imputed_bmi = np.random.normal(mean_bmi, sd)
        return round(imputed_bmi, 2)
    else:
        return None

def impute_height(age, sex):
    mean_height_cm = get_who_stats(age, sex, 'height')
    if mean_height_cm is not None:
        mean_height_inches = mean_height_cm / 2.54  # Convert cm to inches
        return round(mean_height_inches, 2)
    else:
        return None
    
def impute_weight(bmi, height_inches):
    if bmi is not None and height_inches is not None:
        height_meters = height_inches * 0.0254  # Convert inches to meters
        weight_kg = bmi * (height_meters ** 2)
        weight_lbs = weight_kg * 2.20462  # Convert kg to lbs
        return round(weight_lbs, 2)
    else:
        return None
    
def apply_imputation(df):
    def impute_if_missing(row):
        age = row['Basic_Demos-Age']
        sex = row['Basic_Demos-Sex']
        
        if pd.isna(row['Physical-BMI']) and 5 <= age <= 19:
            row['Physical-BMI'] = impute_bmi(age, sex)
        
        if pd.isna(row['Physical-Height']) and 5 <= age <= 19:
            row['Physical-Height'] = impute_height(age, sex)
        
        if pd.isna(row['Physical-Weight']) and row['Physical-BMI'] is not None and row['Physical-Height'] is not None:
            row['Physical-Weight'] = impute_weight(row['Physical-BMI'], row['Physical-Height'])
        
        return row
    
    return df.apply(impute_if_missing, axis=1)

In [ ]:
train_data.head()

In [ ]:
# Apply to datasets
train_data = apply_imputation(train_data)
test_data = apply_imputation(test_data)

# Check the results
print("Number of missing values after imputation:")
print("BMI:", train_data['Physical-BMI'].isna().sum())
print("Height:", train_data['Physical-Height'].isna().sum())
print("Weight:", train_data['Physical-Weight'].isna().sum())

print("\nSample of imputed BMI, Height, and Weight values:")
imputed_sample = train_data[
    (train_data['Physical-BMI'].notnull() | train_data['Physical-Height'].notnull() | train_data['Physical-Weight'].notnull()) &
    ((train_data['Physical-BMI'].notnull() != train_data['Physical-BMI'].notnull().shift()) |
     (train_data['Physical-Height'].notnull() != train_data['Physical-Height'].notnull().shift()) |
     (train_data['Physical-Weight'].notnull() != train_data['Physical-Weight'].notnull().shift()))
].sample(5)[['Basic_Demos-Age', 'Basic_Demos-Sex', 'Physical-BMI', 'Physical-Height', 'Physical-Weight']]
print(imputed_sample)

In [ ]:
# Additional analysis of imputation results
print("\nImputation summary by age:")
age_summary = train_data.groupby('Basic_Demos-Age').agg({
    'Physical-BMI': ['count', 'mean', 'std', 'min', 'max'],
    'Physical-Height': ['count', 'mean', 'std', 'min', 'max'],
    'Physical-Weight': ['count', 'mean', 'std', 'min', 'max'],
    'Basic_Demos-Sex': 'count'
})
print(age_summary)

In [ ]:
# Plot of imputed vs. original data
plt.figure(figsize=(12, 6))
sns.scatterplot(data=train_data, x='Basic_Demos-Age', y='Physical-BMI', hue='Basic_Demos-Sex', alpha=0.5)
plt.title('BMI vs Age (After Imputation)')
plt.savefig('analysis_output/bmi_vs_age_imputed.png')
plt.close()

plt.figure(figsize=(12, 6))
sns.scatterplot(data=train_data, x='Basic_Demos-Age', y='Physical-Height', hue='Basic_Demos-Sex', alpha=0.5)
plt.title('Height vs Age (After Imputation)')
plt.savefig('analysis_output/height_vs_age_imputed.png')
plt.close()

plt.figure(figsize=(12, 6))
sns.scatterplot(data=train_data, x='Basic_Demos-Age', y='Physical-Weight', hue='Basic_Demos-Sex', alpha=0.5)
plt.title('Weight vs Age (After Imputation)')
plt.savefig('analysis_output/weight_vs_age_imputed.png')
plt.close()

print("Imputation analysis plots saved in the 'analysis_output' folder.")

In [ ]:
# Export train_data to CSV
train_output_path = os.path.join(output_folder, 'train_data_imputed.csv')
train_data.to_csv(train_output_path, index=False)
print(f"Imputed train data exported to: {train_output_path}")

# Export test_data to CSV
test_output_path = os.path.join(output_folder, 'test_data_imputed.csv')
test_data.to_csv(test_output_path, index=False)
print(f"Imputed test data exported to: {test_output_path}")

print("Data export completed.")

In [ ]:
# Define categorical columns
category_columns =[
    'Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season',
    'FGC-Season', 'BIA-Season', 'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season',
    'PreInt_EduHx-Season' 
]

# Enumerate categorical columns
def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

for col in category_columns:
    mapping = create_mapping(col, train_data)
    train_data[col] = train_data[col].replace(mapping).infer_objects(copy=False).astype(int)
    test_data[col] = test_data[col].replace(create_mapping(col, test_data)).infer_objects(copy=False).astype(int)

In [ ]:
# High-risk vs. Low-risk analysis
# Suppress RuntimeWarnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# List of columns to drop
columns_to_drop = [
    'FGC-FGC_CU', 'FGC-FGC_PU', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 
    'FGC-FGC_SRL', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSND', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 
    'Fitness_Endurance-Time_Sec', 'stat_41', 'stat_42', 'stat_39', 'stat_92_boxcox', 'stat_92',
    'stat_44'
]

train_data = train_data.drop(columns=columns_to_drop, errors='ignore')

# Separate high-risk and low-risk groups
high_risk = train_data[train_data['sii'] == 3]
low_risk = train_data[train_data['sii'] != 3]

# Function to calculate summary statistics
def get_summary_stats(group):
    return pd.DataFrame({
        'mean': group.mean(),
        'median': group.median(),
        'std': group.std(),
        'min': group.min(),
        'max': group.max()
    })

# List of columns to analyze (excluding 'sii' and non-numeric columns)
columns_to_analyze = train_data.select_dtypes(include=[np.number]).columns.drop('sii')

# Calculate summary statistics for both groups
high_risk_stats = get_summary_stats(high_risk[columns_to_analyze])
low_risk_stats = get_summary_stats(low_risk[columns_to_analyze])

# Calculate the difference in means
mean_diff = high_risk_stats['mean'] - low_risk_stats['mean']

# Perform statistical tests to check for significant differences
def perform_statistical_test(col):
    try:
        # Try t-test first
        t_stat, p_val = stats.ttest_ind(high_risk[col].dropna(), low_risk[col].dropna(), equal_var=False)
        test_type = 't-test'
    except Exception:
        # If t-test fails, use Mann-Whitney U test
        try:
            u_stat, p_val = stats.mannwhitneyu(high_risk[col].dropna(), low_risk[col].dropna(), alternative='two-sided')
            test_type = 'Mann-Whitney U'
        except Exception:
            # If both tests fail, return NaN values
            return pd.Series({'statistic': np.nan, 'p_value': np.nan, 'test_type': 'Failed'})
    
    return pd.Series({'statistic': t_stat if test_type == 't-test' else u_stat, 
                      'p_value': p_val, 
                      'test_type': test_type})

test_results = pd.DataFrame({col: perform_statistical_test(col) for col in columns_to_analyze}).T

# Combine results
comparison_results = pd.concat([
    high_risk_stats.add_prefix('high_risk_'),
    low_risk_stats.add_prefix('low_risk_'),
    mean_diff.rename('mean_difference'),
    test_results
], axis=1)

# Sort by absolute mean difference
comparison_results = comparison_results.sort_values('mean_difference', key=abs, ascending=False)

# Display the top 20 most different features
print(comparison_results.head(20))

In [21]:
# Visualize distributions for top features
def plot_distribution(feature):
    plt.figure(figsize=(10, 6))
    sns.histplot(data=train_data, x=feature, hue='sii', kde=True, common_norm=False)
    plt.title(f'Distribution of {feature} by Risk Group')
    plt.savefig(f'analysis_output/distribution_{feature}.png')
    plt.close()

for feature in comparison_results.head(20).index:
    plot_distribution(feature)

In [ ]:
# Correlation analysis
def get_correlations(group):
    return group.corr().iloc[:, 0].sort_values(key=abs, ascending=False)

high_risk_corr = get_correlations(high_risk[columns_to_analyze])
low_risk_corr = get_correlations(low_risk[columns_to_analyze])

print("\nTop correlations in high-risk group:")
print(high_risk_corr.head(20))
print("\nTop correlations in low-risk group:")
print(low_risk_corr.head(20))

# Calculate correlation differences
corr_diff = high_risk_corr - low_risk_corr
corr_diff = corr_diff.sort_values(key=abs, ascending=False)

print("\nTop correlation differences (high-risk minus low-risk):")
print(corr_diff.head(20))

# Save correlation results to CSV
pd.DataFrame({
    'high_risk_corr': high_risk_corr,
    'low_risk_corr': low_risk_corr,
    'correlation_difference': corr_diff
}).to_csv('analysis_output/correlation_comparison.csv')

# Save results to CSV
comparison_results.to_csv('analysis_output/high_low_risk_comparison.csv')

print("\nAnalysis complete. Results saved to 'analysis_output/high_low_risk_comparison.csv' and 'analysis_output/correlation_comparison.csv'")

In [ ]:
# Visualize top correlation differences
plt.figure(figsize=(16, 12))
corr_diff.head(20).plot(kind='bar')
plt.title('Top 20 Correlation Differences (High-risk minus Low-risk)')
plt.xlabel('Features')
plt.ylabel('Correlation Difference')
plt.tight_layout()
plt.savefig('analysis_output/top_correlation_differences.png')
plt.close()

print("Correlation difference plot saved to 'analysis_output/top_correlation_differences.png'")